In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./.Rhistory
./casia-train-model.ipynb
./remotesensing-13-02468.pdf
./s41598-021-96872-w.pdf
./README.md
./data/.gitignore
./data/Train/Train/Y_train.csv
./data/Train/Train/X_station_train.csv
./data/Train/Train/X_forecast/2D_arpege_20170214.nc
./data/Train/Train/X_forecast/arpege_3D_height_20170214.nc
./data/Train/Train/X_forecast/2D_arome_20170214.nc
./data/Train/Train/X_forecast/arpege_3D_isobar_20170214.nc
./data/Train/Train/Baselines/Baseline_forecast_train.csv
./data/Train/Train/Baselines/Baseline_observation_train.csv
./data/Test/Test/Id_month_test.csv
./data/Test/Test/X_station_test.csv
./data/Test/Test/X_forecast/arpege_3D_isobar_118.nc
./data/Test/Test/X_forecast/2D_arpege_119.nc
./data/Test/Test/X_forecast/arpege_3D_height_119.nc
./data/Test/Test/X_forecast/2D_arome_119.nc
./data/Test/Test/Baselines/Baseline_forecast_test.csv
./data/Test/Test/Baselines/Baseline_observation_test.csv
./data/Other/Other/NW_masks.nc
./data/Other/Other/stations_coordinates.csv
./.git/COMMIT_EDITMS

In [3]:
### Fonctions utiles :
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

### load X_train

In [4]:
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)
coords_fname  = './data/Other/Other/stations_coordinates.csv'
fname = './data/Train/Train/X_station_train.csv'

In [6]:
# imputer = KNNImputer(n_neighbors=1)
# imputer.fit(df_imput)
# df_imput = train[["ff","t","td","hu","dd","precip"]].copy()
# X = imputer.transform(df_imput)
# print("Je commence l'imputation")
# df_imput= pd.DataFrame(X,columns = df_imput.columns)
# train[["ff","t","td","hu","dd","precip"]] = df_imput[["ff","t","td","hu","dd","precip"]]
# # reorganisation des données.
# print("Fin de l'imputation")

In [7]:
from sklearn.impute import KNNImputer

In [8]:
# Cette fonction ne prend en compte que les données présentes dans Xstation
def load_X_train(fname,coords_fname) :
    coords = pd.read_csv(coords_fname)
    train = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    train['number_sta']=train['number_sta'].astype('category')
    #sort by station, then by date 
    train = train.sort_values(['number_sta','date'])   
    method_traitements = ["mean","min","max","std","median","sum",percentile(10),percentile(25),percentile(80),percentile(95)]
    dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                     ,"hu":method_traitements,"dd":method_traitements,"precip":"sum"}
    Base_obs = train[{"number_sta","date","ff","t","td","hu","dd","precip"}].copy()
    Base_obs.set_index('date',inplace = True)  

    # type de traitement.
    Base_obs_type = Base_obs.groupby('number_sta').resample('D').agg(dict_to_split)
    Base_obs_type.columns = ['_'.join(col).strip() for col in Base_obs_type.columns.values]
    Base_obs_type = Base_obs_type.reset_index(['date','number_sta'])
    Base_obs_type['number_sta'] = Base_obs_type['number_sta'].astype('category')   
    #Select the observations the day before

    Base_obs_type['Prediction'] = Base_obs_type['precip_sum'].shift(1)
    Base_obs_type = Base_obs_type.sort_values(by=["number_sta","date"])
    #del Base_obs['precip']
    #Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})
    del Base_obs_type["precip_sum"]
    date = first_date
    dates = []
    while date <= (last_date - datetime.timedelta(days=1)):
        date += datetime.timedelta(days=1)
        dates.append(date)

    d_dates =  pd.DataFrame(dates, columns = ['date'])
    d_dates['day_index'] = d_dates.index
    y_f = Base_obs_type.merge(d_dates,how="right",on = ["date"])
    y_f = y_f[y_f['date']!=last_date]
    y_f['Id'] = y_f[list(y_f.columns)[0]].astype(str) + '_' + \
                      y_f['day_index'].astype(str) 
    #y_f["number_sta"] = y_f[("number_sta","")]
    y_f = y_f.merge(coords,on=["number_sta"],how="right")
    X_train = y_f.copy()#drop(["day_index"],axis=1)
    X_train["month"] = X_train["date"].dt.month
    X_train["day"] = X_train["date"].dt.day
    X_train.drop(["date"],axis=1,inplace=True)
    return X_train

In [9]:
train = load_X_train(fname,coords_fname)

In [10]:
train_forecast = pd.read_csv("./data/Train/Train/Baselines/Baseline_forecast_train.csv")

In [11]:
train_forecast.rename(columns={"Prediction":"Forecasted"},inplace = True)
train_forecast.drop(["date","number_sta"],axis = 1 ,inplace = True)

In [12]:
value_list = list(train["Id"])
boolean_series = train_forecast.Id.isin(value_list)
train_forecast = train_forecast[boolean_series]

In [13]:
train = train.merge(train_forecast,on=["Id"],how="right")

### Load Y_test

In [14]:
fname_test = "./data/Test/Test/X_station_test.csv"
coords_fname_test = './data/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname_test)
def load_X_test(fname_test,coords_fname_test) :
    test = pd.read_csv(fname_test)
    test["Id"] =test["Id"].str.rsplit("_",n=1,expand=True)[0]
    month = test["month"]
    test.drop(["precip","month"],axis = 1,inplace = True)
    method_traitements = ["mean","min","max","std","median","sum",percentile(10),percentile(25),percentile(80),percentile(95)]
    dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                     ,"hu":method_traitements,"dd":method_traitements}
    test["Id"] = test["Id"].astype("category")
    Base_obs = test[{"Id","ff","t","td","hu","dd"}].copy()
    # type de traitement.
    Base_obs_type = Base_obs.groupby('Id').agg(dict_to_split)
    Base_obs_type = Base_obs_type.sort_values(by=["Id"])
    test = Base_obs_type.copy()
    test.columns = ['_'.join(col).strip() for col in test.columns.values]
    test = test.reset_index(['Id'])
    base_test = pd.read_csv("./data/Test/Test/Baselines/Baseline_observation_test.csv")
    test = test.merge(base_test,on=["Id"],how="right")
    test[["number_sta","day_index"]] = test["Id"].str.split("_",expand=True)
    test["number_sta"] = test["number_sta"].astype(int)
    test = test.merge(coords,on="number_sta",how='right')
    date = first_date
    dates = []
    while date <= (last_date - datetime.timedelta(days=1)):
        date += datetime.timedelta(days=1)
        dates.append(date)

    d_dates =  pd.DataFrame(dates, columns = ['date'])
    d_dates['day_index'] = d_dates.index
    d_dates["month"]=month
    d_dates["day"] = d_dates["date"].dt.day
    del d_dates["date"]
    test.dropna(subset=["day_index"],inplace=True)
    test["day_index"] = test["day_index"].astype(int)
    test = test.merge(d_dates,on=["day_index"],how="right")
    test = test.dropna(subset=["lat"])
    return test


In [15]:
test = load_X_test(fname_test,coords_fname_test)

In [16]:
test_forecast = pd.read_csv("./data/Test/Test/Baselines/Baseline_forecast_test.csv")

In [17]:
test_forecast.rename(columns={"Prediction":"Forecasted"},inplace = True)
value_list = list(test["Id"])
boolean_series = test_forecast.Id.isin(value_list)
test_forecast = test_forecast[boolean_series]
test = test_forecast.merge(test,on=["Id"],how="right")

In [18]:
forcasted = list(test["Forecasted"])
del test["Forecasted"]
test["Forecasted"] = forcasted


In [19]:
test["Forecasted"] = test["Forecasted"].fillna(test["Prediction"])

### Load Y

In [20]:
fname = './data/Train/Train/Y_train.csv'
param = 'Ground_truth'  #weather parameter name in the file ('Ground_truth' about Y and 'Prediction' about baseline)
ytrain= pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')

In [21]:
ytrain = ytrain[ytrain["date"]!=last_date].copy()

In [22]:
### Methodologie drop_all_na
X_full = train.dropna()
Y_full = ytrain.dropna(subset=["Ground_truth"])
#X_full =X_full[X_full.index.isin(Y_full.index)].dropna()
X_full = X_full.sort_values("Id")
Y_full = Y_full.sort_values("Id")
value_list = list(X_full["Id"])
boolean_series = Y_full.Id.isin(value_list)
Y_full = Y_full[boolean_series]
value_list = list(Y_full["Id"])
boolean_series = X_full.Id.isin(value_list)
X_full = X_full[boolean_series]

In [ ]:
X_full = train.copy()

In [ ]:
### Methodologie drop_na from Ytrain
Y_full = ytrain.dropna(subset=["Ground_truth"])

In [ ]:
value_list = list(Y_full["Id"])
boolean_series = X_full.Id.isin(value_list)
X_full = X_full[boolean_series]

In [ ]:
train.columns

In [23]:
to_drop = ["Id","number_sta","day_index","day"]#,'lat', 'lon', 'height_sta']

In [24]:
Ximp = X_full.drop(to_drop,axis=1).values#],"lon","lat","height_sta"
Yimp = np.array(Y_full["Ground_truth"])

In [ ]:
### pour l'ensemble test
x_full = test.copy()

In [ ]:
ximp = x_full.drop(["Id","number_sta","day_index"],axis=1).values

In [ ]:
Xfull = Ximp.copy()

### Application du modèle.

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
### réaliser une imputation des données manquantes.
imputer = KNNImputer(n_neighbors=1)
imputer.fit(ximp)

In [ ]:
ximp = imputer.transform(ximp)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
imp.fit(Ximp)

In [ ]:
ximp=imp.transform(ximp)

### Fonction MAPE

In [25]:
def MAPE(yp,yt) :
    taille = len(yp)
    mape = 0
    for i in range(taille) :
        mape += abs((yt[i]-yp[i])/(yt[i]+1))
    mape *= 100/taille
    return mape

In [26]:
def MAPE2(Y_Predicted,Y_actual):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/(Y_actual+1)))*100
    return mape

In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [28]:
scaler = StandardScaler()

In [29]:
scaler.fit(Ximp)
Ximp = scaler.transform(Ximp)

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
...     Ximp, Yimp, test_size=0.33, random_state=42)

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
reg.score(X_train, y_train)

In [ ]:
y_predicted = reg.predict(X_train)

In [ ]:
y_predicted = [elt if elt >=0 else 0 for elt in y_predicted ]

In [ ]:
MAPE(y_predicted,y_train)

In [ ]:
y_predicted = reg.predict(X_test)

In [ ]:
y_predicted = [elt if elt >=0 else 0 for elt in y_predicted ]

In [ ]:
MAPE(y_predicted,y_test)

### RandomForestRegression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train_plus = np.concatenate([np.array(X_train),np.array(y_predicted_train_smv).reshape(len(X_train),1)] , axis=1)

In [ ]:
regr = RandomForestRegressor()
regr.fit(X_train_plus, y_train)

In [ ]:
regr.score(X_train_plus,y_train)

In [ ]:
y_predicted_rf_train = regr.predict(X_train_plus)
#y_predicted_rf = [elt if elt >=0 else 0 for elt in y_predicted ]

In [ ]:
MAPE(y_predicted_rf_train  ,y_train)

In [ ]:
X_test_plus = np.concatenate([np.array(X_test),np.array(y_predicted_svm_test).reshape(len(X_test),1)] , axis=1)

In [ ]:
y_predicted_rf_test = regr.predict(X_test_plus)
#y_predicted = [elt if elt >=0 else 0 for elt in y_predicted ]

In [ ]:
MAPE(y_predicted_rf_test ,y_test)

### SVM

In [41]:
from sklearn import svm

In [43]:
X_train_plus = np.concatenate([np.array(X_train),np.array(y_predicted_sgdr_train).reshape(len(X_train),1)] , axis=1)

In [53]:
regr_svm = svm.SVR()
regr_svm.fit(Ximp,Yimp)

SVR()

In [45]:
regr_svm.score(X_train_plus,y_train)

0.6329125860292935

In [46]:
y_predicted_svm_train = regr_svm.predict(X_train_plus)
#y_predicted = [elt if elt >=0 else 0 for elt in y_predicted ]

In [47]:
MAPE(y_predicted_svm_train ,y_train)

26.57007017294683

In [48]:
X_test_plus = np.concatenate([np.array(X_test),np.array(y_predicted_sgdr_test).reshape(len(X_test),1)] , axis=1)

In [49]:
y_predicted_svm_test= regr_svm.predict(X_test_plus)
#y_predicted = [elt if elt >=0 else 0 for elt in y_predicted ]

In [50]:
MAPE(y_predicted_svm_test ,y_test)

27.433196749654613

In [54]:
y_predicted_all = regr_svm.predict(Ximp)
#y_predicted_all = [elt if elt >=0 else 0 for elt in y_predicted_all ]


In [ ]:
len(y_predicted_all)

In [ ]:
#idée refaire une projection et faire de tels sorte que cela soit smooth

In [ ]:
MAPE(y_predicted_all,Yimp)

In [ ]:
MAPE2(y_predicted_all,Yimp)

In [ ]:
len(y_predicted_all)

In [ ]:
26.714607117563933,27.857640502040667

### SGD_REGRESSOR

In [32]:
from sklearn.linear_model import SGDRegressor

In [33]:
regr_sgdr = SGDRegressor()
regr_sgdr.fit(X_train,y_train)

SGDRegressor()

In [34]:
regr_sgdr.score(X_train,y_train)

0.5978431555598802

In [36]:
y_predicted_sgdr_train = regr_sgdr.predict(X_train)
y_predicted_sgdr_train = [elt if elt >=0 else 0 for elt in y_predicted_sgdr_train ]

In [37]:
MAPE( y_predicted_sgdr_train ,y_train)

47.61973642527675

In [39]:
y_predicted_sgdr_test = regr_sgdr.predict(X_test)
y_predicted_sgdr_test = [elt if elt >=0 else 0 for elt in y_predicted_sgdr_test ]

In [40]:
MAPE(y_predicted_sgdr_test ,y_test)

47.91342601658667

### Lasso

In [ ]:
from sklearn import linear_model

In [ ]:
reg_lasso = linear_model.Lasso(alpha=0.1)
reg_lasso.fit(X_train,y_train)

In [ ]:
reg_lasso.score(X_train,y_train)

In [ ]:
MAPE(reg_lasso.predict(X_train),y_train)

In [ ]:
MAPE(reg_lasso.predict(X_test),y_test)

### MLP-NN 

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
regr_mlpr = MLPRegressor(random_state=1, max_iter=1000).fit(X_train, y_train)

In [ ]:
regr_mlpr.score(X_train,y_train)

In [ ]:
MAPE(regr_mlpr.predict(X_train),y_train)

In [ ]:
MAPE(regr_mlpr.predict(X_test),y_test)

### XG-Boost

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
reg_xgBoost = GradientBoostingRegressor()
reg_xgBoost.fit(X_train, y_train)

In [ ]:
reg_xgBoost.score(X_train,y_train)

In [ ]:
MAPE(reg_xgBoost.predict(X_train),y_train)

In [ ]:
MAPE(reg_xgBoost.predict(X_test),y_test)

### Gérer les données manquantes

In [55]:
Xna = train[~train.index.isin(train.dropna().index)]

In [56]:
# pour chaque station présente dans le Xna
Xna_reduit = Xna[["Id","lat","lon","height_sta","day_index"]].copy()
Xna_reduit["Prediction"] = None
Xc_reduit = X_full[["Id","lat","lon","height_sta","day_index"]].copy()
Xc_reduit["Prediction"] = list(y_predicted_all)
Xna_reduit.set_index("Id",inplace = True)
Xc_reduit.set_index("Id",inplace = True)

In [57]:
from sklearn.impute import KNNImputer

In [58]:
df_feed = pd.concat([Xc_reduit,Xna_reduit],ignore_index=True)

In [59]:
### réaliser une imputation des données manquantes.
imputer = KNNImputer(n_neighbors=1)
imputer.fit(df_feed)

KNNImputer(n_neighbors=1)

In [60]:
result = imputer.transform(Xna_reduit)

In [61]:
df = pd.DataFrame()
df["Id"] = ytrain.dropna(subset = ["Ground_truth"])["Id"]
df["Prediction"] = ytrain.dropna(subset = ["Ground_truth"])["Ground_truth"]

In [62]:
Xna["result"] = result[:,4]

In [63]:
Xna = Xna.merge(df,on=["Id"],how="right").dropna(subset=["lat"])

In [64]:
MAPE(list(Xna["result"]),list(Xna["Prediction_y"]))

45.32125416904505

In [65]:
Xasc = Xna[["lat","lon","height_sta","Forecasted","result","month","Prediction_x"]].copy().values

In [66]:
Yasc = np.array(Xna["Prediction_y"])

In [67]:
Xasc = StandardScaler().fit_transform(Xasc)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
...     Xasc, Yasc, test_size=0.33, random_state=42)

In [69]:
# modele to complete missing data 
regr_svm_na = svm.SVR()
regr_svm_na.fit(Xasc,Yasc)

SVR()

In [86]:
MAPE(regr_svm_na.predict(Xasc),Yasc)

30.348130357137734

In [ ]:
xasc = Xna[["lat","lon","height_sta","Forecasted","result","month","day","Prediction_x"]].copy()
xasc["Forecasted"] = list(xasc["Forecasted"])[0:50000] + [None]*(71829-50000)

In [ ]:
xasc["Forecasted"] = xasc["Prediction_x"]

In [ ]:
xasc_v = xasc.values

In [ ]:
xasc_v = StandardScaler().fit_transform(xasc_v)

In [ ]:
MAPE(regr_svm_na.predict(xasc_v),Yasc)

In [ ]:
MAPE2(regr_svm_na.predict(xasc_v),Yasc)

In [ ]:
MAPE(list(Xna["Forecasted"]),list(Xna["Prediction_y"]))

### Prédire la partie test:

#### Sans nan 

In [70]:
X_full_tp = test.dropna() # tp stands for to predict
id_predicted_1 = list(X_full_tp["Id"])
Ximp_tp = X_full_tp.drop(to_drop,axis=1).values
Ximp_tp = StandardScaler().fit_transform(Ximp_tp)
y_predicted_1 = list(regr_svm.predict(Ximp_tp))

#### Avec nan 

In [71]:
Xna_tp = test[~test.index.isin(test.dropna().index)]

In [72]:
# pour chaque station présente dans le Xna
Xna_reduit_tp = Xna_tp[["Id","lat","lon","height_sta","month"]].copy()
id_predicted_2 = list(Xna_tp["Id"])
Xna_reduit_tp["Prediction"] = None
Xna_reduit_tp.set_index("Id",inplace = True)

In [73]:
Xc_reduit_tp = X_full_tp[["Id","lat","lon","height_sta","month"]].copy()
Xc_reduit_tp["Prediction"] = list(y_predicted_1)
Xc_reduit_tp.set_index("Id",inplace = True)

In [74]:
imputer = KNNImputer(n_neighbors=1).fit(Xc_reduit_tp)

In [75]:
result_tp = imputer.transform(Xna_reduit_tp)

In [76]:
Xna_tp["result"] = result_tp[:,4]

In [77]:
Xasc_tp = Xna_tp[["lat","lon","height_sta","Forecasted","result","month","Prediction"]].copy().values

In [78]:
Xasc_tp = StandardScaler().fit_transform(Xasc_tp)

In [79]:
y_predicted_2 = list(Xna_tp["Forecasted"])#list(regr_svm_na.predict(Xasc_tp))

In [ ]:
backup_1 = 0.5 * np.array(X_full_tp["Forecasted"]) + 0.5 * np.array(X_full_tp["Prediction"])
backup_2 = 0.5 * np.array(Xna_tp["Forecasted"]) + 0.5 * np.array(Xna_tp["Prediction"])

In [80]:
y_pred_1 = [e1 if e1>=0 else 0 for e1 in y_predicted_1]
y_pred_2 = [e1 if e1>=0 else 0 for e1 in y_predicted_2]

In [81]:
df1 = pd.DataFrame()
df1["Id"] = id_predicted_1
df1["Prediction"] = y_pred_1

In [82]:
df2 = pd.DataFrame()
df2["Id"] = id_predicted_2
df2["Prediction"] = y_pred_2

In [83]:
df_submit = pd.concat([df1,df2],ignore_index=True)

### Partie submission

In [ ]:
y_predicted = list(regr_svm.predict(ximp))
id_predicted = list(test["Id"])

In [ ]:
df_submit = pd.DataFrame()
df_submit["Id"] = id_predicted
df_submit["Prediction"] = y_predicted

#### enregistrement

In [84]:
df_submit["Prediction"] = df_submit.Prediction.apply(lambda s : s + 1)

In [85]:
df_submit.to_csv('./working/' + "submission.csv",index=False)

In [ ]:
df_submit.shape

#### reload

In [ ]:
df_submit = pd.read_csv("./working/submission.csv")

In [ ]:
df_submit